# Deep Learning-Based WWR and Floor Count Extraction from Façade Images to Improve UBEM

CISBAT 2025

[Ayca Duran](https://systems.arch.ethz.ch/ayca-duran), [Panagiotis Karapiperis](https://www.linkedin.com/in/panagiotis-karapiperis-ethz/), [Christoph Waibel](https://systems.arch.ethz.ch/christoph-waibel), [Arno Schlueter](https://systems.arch.ethz.ch/arno-schlueter)

### WWR Extraction Workflow

This notebook performs the WWR extraction from the rectified images. It uses 3 inputs: the facade masks that are predicted from original images and rectified together with the images, and the window predictions from the FCN and ground-SAM models from the already rectified images.

In [7]:
# Import libraries
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from scipy.cluster.hierarchy import fcluster, linkage

In [8]:
### Input Paths
images_path = "example/rectified/images"
facade_pred_path = "example/rectified/facades"
fcn_window_pred_path = "example/predictions/fcn_resnet50_rectified"
groundsam_window_pred_path = "example/predictions/gsam_rectified"

# Save Paths
clustered_windows_path = "example/WWR/clustered_windows"
clustered_facade_path = "example/WWR/clustered_facade"
WWR_path = "example/WWR/wwr_per_level"

if not os.path.exists(clustered_windows_path):
    os.makedirs(clustered_windows_path, exist_ok=True)
if not os.path.exists(clustered_facade_path):
    os.makedirs(clustered_facade_path, exist_ok=True)
if not os.path.exists(WWR_path):
    os.makedirs(WWR_path, exist_ok=True)

In [9]:
### Function to merge facade regions that belong to the same floor
def merge_regions_by_lowest_point(labels):

    # Get unique labels (ignoring background label 0)
    unique_labels = np.unique(labels)
    unique_labels = unique_labels[unique_labels != 0]

    # Dictionary to store the lowest points of each label
    lowest_points = {}

    # Find the lowest point for each region (the one with the maximum y-coordinate)
    for label in unique_labels:
        y_indices, x_indices = np.where(labels == label)
        lowest_y = np.max(y_indices)
        lowest_points[label] = lowest_y
    
    # Now, merge regions that have the same lowest y-coordinate
    label_mapping = {}  # To map original labels to new labels
    new_label = 1  # Start from label 1 (since 0 is reserved for background)
    
    # Sort regions by their lowest y-coordinate
    sorted_labels = sorted(lowest_points.items(), key=lambda x: x[1])

    for i, (label, _) in enumerate(sorted_labels):
        if label not in label_mapping:
            # Assign a new label to the region
            label_mapping[label] = new_label
            new_label += 1
        
        # Check for regions with the same lowest y-coordinate and merge them
        for j in range(i + 1, len(sorted_labels)):
            label2, _ = sorted_labels[j]
            if lowest_points[label] == lowest_points[label2]:
                label_mapping[label2] = label_mapping[label]

    # Create the merged label matrix
    merged_labels = np.copy(labels)
    for old_label, new_label in label_mapping.items():
        merged_labels[merged_labels == old_label] = new_label
    
    return merged_labels

In [10]:
### Function to fix the clustering of the windows to go from top to bottom
def reassign_cluster_labels(clusters):
    return_clusters = []
    return_clusters.append(1)
    index = 1
    for i in range(1,len(clusters)):
        if clusters[i] == clusters[i-1]:
            return_clusters.append(index)
        else:
            index = index+1
            return_clusters.append(index)
    return_clusters = np.array(return_clusters)
    return return_clusters

In [11]:
def calculate_WWR(facade_mask, windows_mask):
    # Count the number of 1s in each mask
    facade_sum = np.sum(facade_mask)
    windows_sum = np.sum(windows_mask)
    
    # Avoid division by zero by checking if facade_sum is zero
    if facade_sum == 0:
        return 0  # or return a message like "Facade mask has no pixels"
    
    # Calculate the ratio of window pixels to facade pixels
    ratio = windows_sum / facade_sum
    return ratio

## Loop WWR Extraction

In [ ]:
# Create DF to save results
n = 6  # 6 WWR columns
columns = ['filename'] + ["WWR_all"] + [f'WWR_Level_{i}' for i in range(0, n)]
df = pd.DataFrame(columns=columns)

# Iterate through images
for index, filename in enumerate(os.listdir(images_path)):

    ## 1. Load image and masks
    image = PIL.Image.open(os.path.join(images_path, filename))
    image = np.array(image)

    # facade
    facade_prediction = PIL.Image.open(os.path.join(facade_pred_path, filename.split(".jpg")[0]+".png")).convert("L")
    facade_prediction = np.array(facade_prediction)

    # windows
    fcn_window_prediction = PIL.Image.open(os.path.join(fcn_window_pred_path, filename.split(".jpg")[0]+".png")).convert("L")
    fcn_window_prediction = np.array(fcn_window_prediction)
    groundsam_window_prediction = PIL.Image.open(os.path.join(groundsam_window_pred_path, filename.split(".jpg")[0]+".png")).convert("L")
    groundsam_window_prediction = np.array(groundsam_window_prediction)
    window_prediction = np.logical_or(fcn_window_prediction, groundsam_window_prediction).astype(np.uint8)
    window_prediction = window_prediction * facade_prediction # keep only windows that are within facade

    ## 2. Crop around facade bounding box
    y_indices, x_indices = np.where(facade_prediction > 0)
    x_min, x_max = x_indices.min(), x_indices.max()
    y_min, y_max = y_indices.min(), y_indices.max()

    image = image[y_min:y_max+1, x_min:x_max+1]
    facade_prediction = facade_prediction[y_min:y_max+1, x_min:x_max+1]
    window_prediction = window_prediction[y_min:y_max+1, x_min:x_max+1]

    ## 3. Prepare and cluster windows
    kernel = np.ones((15, 15), np.uint8)  # Kernel size can be adjusted
    window_prediction = cv2.morphologyEx(window_prediction, cv2.MORPH_OPEN, kernel)
    window_prediction_rect = cv2.morphologyEx(window_prediction, cv2.MORPH_OPEN, kernel)

    ### Identify instances of windows with connected components
    num_labels, labels = cv2.connectedComponents(window_prediction)
    unique_labels = np.unique(labels)
    unique_labels = unique_labels[unique_labels != 0]
    # Count distinct regions
    # Note: num_labels includes the background as label 0, so subtract 1 for the count of class 1 regions
    num_instances = num_labels - 1
    print(num_instances)

    # Visualize rectified image and facade / windows predictions
    fig, axes = plt.subplots(1, 3, figsize=(20, 10), sharex=True)
    axes[0].imshow(image)
    axes[0].set_title("Rectified Image")
    axes[0].grid(True)
    axes[1].imshow(facade_prediction, cmap="rainbow", alpha= 0.5)
    axes[1].imshow(window_prediction, cmap="rainbow", alpha= 0.5)
    axes[1].set_title("Facade & Windows Predictions")
    axes[1].grid(True)

    ### Find uppermost Point for each window
    windows_uppermost_points = {}
    for label in unique_labels:
        class_positions = np.argwhere(labels == label)
        uppermost_pixel = class_positions[np.argmin(class_positions[:, 0])]
        windows_uppermost_points[label] = tuple(uppermost_pixel)

    axes[0].imshow(image)
    axes[0].set_title("Windows Uppermost Points")
    # Plot the uppermost points
    for label, uppermost_pixel in windows_uppermost_points.items():
        y, x = uppermost_pixel  # Note: matplotlib uses x, y coordinates
        axes[2].plot(x, y, "bo")  # 'go' means green circles
        axes[2].text(x + 5, y, f"Window {label}", color="white", fontsize=10, bbox=dict(facecolor='black', alpha=0.5))

    ### Cluster Windows into floor groups 
    points = np.array(list(windows_uppermost_points.values()))
    # Use only the first coordinate (row) for clustering
    rows = points[:, 0].reshape(-1, 1)
    # Perform hierarchical clustering
    try:
        linked = linkage(rows, method='ward')  # Use Ward's method for clustering
    except:
        continue
    clusters = fcluster(linked, t=80, criterion='distance')  # t=50 defines max distance between clusters
    ## Sort clusters to ensure starting from top to bottom
    clusters = reassign_cluster_labels(clusters)

    # Group points by clusters
    clustered_points = {}
    for idx, cluster_id in enumerate(clusters):
        clustered_points.setdefault(cluster_id, []).append(list(windows_uppermost_points.values())[idx])

    # Group windows with clusters
    # Initialize a new mask with the same shape as the `labels` array
    clustered_mask = np.zeros_like(labels)
    # Map each instance label in `labels` to its cluster index
    for instance_label, cluster_index in enumerate(clusters, start=1):  # start=1 to match labels
        clustered_mask[labels == instance_label] = cluster_index   # +1 to keep 0 for background
    axes[2].imshow(clustered_mask, cmap="rainbow", alpha= 0.5)
    axes[2].set_title("Clustered Windows")
    axes[2].grid(True)
    plt.savefig(os.path.join(clustered_windows_path, filename), dpi=300, bbox_inches='tight') # Save clustered windows fig  
    plt.show

    unique_labels2 = np.unique(clustered_mask)
    unique_labels2 = unique_labels2[unique_labels2 != 0]

    uppermost_points_floors = {}
    # Calculate the center of each labeled region
    for label in unique_labels2:
        class_positions = np.argwhere(clustered_mask == label)
        uppermost_pixel = class_positions[np.argmin(class_positions[:, 0])]
        uppermost_points_floors[label] = tuple(uppermost_pixel)

    # Create a blank white image
    image_height, image_width = image.shape[0], image.shape[1]  # Adjust as needed

    lines = []

    # Iterate through the points and draw horizontal lines
    for point_id, (y, x) in uppermost_points_floors.items():
        # Draw a horizontal line across the entire image at the y-coordinate
        cv2.line(image, (0, y), (image_width - 1, y), color=(0, 0, 255), thickness=2)
        lines.append((0, y, image_width - 1, y))

    ##### Split facade
    # Step 2: Create a canvas to draw the lines
    line_canvas = np.zeros_like(facade_prediction, dtype=np.uint8)

    for i, (x1, y1, x2, y2) in enumerate(lines, start=2):  # Start labeling from 2
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Convert to integers
        cv2.line(line_canvas, (x1, y1), (x2, y2), color=255, thickness=8)

    # Step 3: Combine the mask and lines
    combined = facade_prediction.copy()
    combined[line_canvas > 0] = 0  # Remove mask where lines exist

    num_labels_fac, regions = cv2.connectedComponents(combined)
    regions = merge_regions_by_lowest_point(regions)
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True)
    axes[0].imshow(image)
    axes[0].imshow(combined, cmap = "BrBG", alpha=0.5)
    axes[0].set_title("Facade Prediction & Floor Lines")
    for line in lines:
        (x1, y1, x2, y2) = line

    axes[1].imshow(regions, cmap="BrBG", alpha=0.5)
    axes[1].imshow(clustered_mask, cmap="rainbow", alpha= 0.5)
    axes[1].set_title("Clustered Facade")
    axes[1].grid(True)
    plt.savefig(os.path.join(clustered_facade_path, filename), dpi=300, bbox_inches='tight')
    plt.show

    ### Calculate and plot per level WWR
    row = {'filename': filename}
    row["WWR_all"] = round(calculate_WWR(facade_prediction, window_prediction), 3)

    i = 0
    fig, axes = plt.subplots(1, (len(np.unique(regions))-2), figsize=(20, 10), sharex=True)
    for target_class in range(1, (len(np.unique(regions))-1)):
        focused_facade = (regions == target_class+1)
        focused_windows = (clustered_mask == target_class)

        # Calculate WWR
        count_facade = np.sum(focused_facade == 1)
        count_windows = np.sum(focused_windows == 1)
        WWR = count_windows / (count_windows + count_facade)
        WWR = round(WWR, 3)
        row[f'WWR_Level_{(len(np.unique(regions))-2-1)-i}'] = WWR # -1 because ground floor is WWR_0
        
        if type(axes) != np.ndarray:
            axes = [axes]

        axes[i].imshow(image)
        axes[i].imshow(focused_facade, cmap="gray", alpha=0.6)
        axes[i].imshow(focused_windows, cmap="rainbow", alpha=0.6)
        axes[i].set_title(f"WWR = {WWR}")
        i = i + 1

        if target_class == (len(np.unique(regions))-2):
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
            

    df.loc[index] = row
    plt.savefig(os.path.join(WWR_path, filename), dpi=300, bbox_inches='tight')
    plt.close()
    #break

# Save WWR data to CSV file
df.to_csv('example/WWR/wwr_from_visual_data.csv')